In [23]:
import torch
from torch import optim
from torch import nn
from pytorch_pretrained_bert import TransfoXLModel, TransfoXLCorpus, TransfoXLTokenizer
import time, math, sys
sys.path.append('../')
import numpy as np

import models
from utils.data_loader import JSONFileDataLoader
from utils.transf_data_loader import JSONFileDataLoaderTransf
from utils.framework import FewShotREFramework, FewShotREModel
from models.sentence_encoding.basic_sentence_encoder import CNNSentenceEncoder
from models.proto_model import Proto

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

from pytorch_openai_transformer_lm.model_pytorch import TransformerModel
from pytorch_openai_transformer_lm.model_pytorch import load_openai_pretrained_model, DEFAULT_CONFIG
from pytorch_openai_transformer_lm.text_utils import TextEncoder

In [24]:
class TransformerSentenceEncoder(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.args = DEFAULT_CONFIG
        self.model = TransformerModel(self.args)
        load_openai_pretrained_model(self.model, path='../finetune_transformer_lm/model/',
                                           path_names='../pytorch_openai_transformer_lm/')

    def forward(self, inputs):
        word = inputs.to(self.device)
        print(word.shape)
        h = self.model(word)
        print(h.shape)
        return h[-1]

In [3]:
model_name = 'proto'
N = 5
K = 5
max_length = 40
train_data_loader = JSONFileDataLoaderTransf('../../FewRel/data/train.json',
                                             max_length=max_length, cuda=False)
val_data_loader = JSONFileDataLoaderTransf('../../FewRel/data/val.json',
                                           max_length=max_length, cuda=False)

Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing
Loading data file...
Finish loading
Elimiating case sensitive problem...
Finish eliminating
Finish building
Pre-processing data...
Finish pre-processing


In [4]:
framework = FewShotREFramework(train_data_loader, val_data_loader, None)
encoder = TransformerSentenceEncoder()
model = Proto(encoder)

Loading weights...


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [14]:
%%time
support, query, label = train_data_loader.next_batch(8, 5, 5, 5)
print(support.max())
support[support >= 40000] = 1

tensor(267739)
CPU times: user 11.1 ms, sys: 1.22 ms, total: 12.3 ms
Wall time: 10.6 ms


In [15]:
%%time
encoder(support)

torch.Size([200, 40])
torch.Size([1, 200, 768])
CPU times: user 33.9 ms, sys: 41.4 ms, total: 75.2 ms
Wall time: 77.9 ms


tensor([[-0.9686, -1.3646, -0.7122,  ...,  0.9607, -0.0957, -1.0857],
        [-0.2078, -2.1590, -0.8007,  ...,  0.5149,  0.2589,  0.2698],
        [ 0.1536, -1.7496, -1.4168,  ...,  1.0634,  0.5634,  0.0037],
        ...,
        [ 0.0691, -0.5932, -0.1879,  ..., -0.5596, -1.2896,  0.0753],
        [ 0.7865,  0.2918, -0.5410,  ..., -0.2975, -0.5084,  0.3388],
        [ 0.5703, -0.3014, -0.5006,  ..., -0.8791,  0.2266,  0.5928]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [18]:
import json
text_encoder = json.load(open('../finetune_transformer_lm/model/encoder_bpe_40000.json'))

In [ ]:
TextEncoder('../finetune_transformer_lm/model/encoder_bpe_40000.json')